In [1]:
import tensorflow as tf
from time import time
import numpy
import os
import json
import pickle
import pandas
import datetime
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import data_pipeline
import conv_model
import initialize
import prepare_data
import flacdb
import plot_batch

In [2]:
%%time

H = initialize.load_hypes()
initial_data_path = '/scr1/mimic/initial_data/'

# sig_data, metadata = initialize.load_initial_data(save_path=initial_data_path)
sig_data, metadata = initialize.load_initial_data(load_path=initial_data_path)

diagnosis = initialize.load_diagnosis(H['icd_codes'], metadata)
diagnosis = initialize.augment_diagnosis(diagnosis, metadata)
diagnosis = initialize.fix_diagnosis(diagnosis)
diagnosis = initialize.conform_diagnosis(diagnosis, metadata)

partition = initialize.load_partition(H, sig_data)

CPU times: user 3.49 s, sys: 268 ms, total: 3.76 s
Wall time: 3.76 s


In [4]:
%%time
(diagnosis[partition['validation']] == 1).any(level=0, axis=0).sum()

CPU times: user 72 ms, sys: 4 ms, total: 76 ms
Wall time: 71.9 ms


25000                  117
2724                   143
2762                    93
27651                   16
27652                   18
27800                   22
2859                    60
4019                   236
41071                   42
41401                  119
4240                    28
4241                    27
42731                  152
42732                   16
4275                    25
4280                   143
42832                    8
42833                   16
431                     32
5119                    35
51881                  136
5715                    24
5849                   111
5859                    52
78552                   65
79902                   15
99592                   92
gender_F               228
gender_M               371
race_asian              10
race_black              51
race_hispanic           29
race_white             382
age_at_least_75        156
height_at_least_70     115
weight_at_least_100    108
died                    97
d

In [5]:
%%time
(diagnosis[partition['validation']] == 1).any(level=0, axis=0).sum()

CPU times: user 76 ms, sys: 0 ns, total: 76 ms
Wall time: 71.1 ms


25000                  102
2724                   133
2762                    82
27651                   12
27652                   14
27800                   22
2859                    73
4019                   240
41071                   23
41401                  103
4240                    20
4241                    18
42731                  138
42732                   17
4275                    35
4280                   129
42832                   14
42833                   14
431                     27
5119                    40
51881                  124
5715                    21
5849                   107
5859                    52
78552                   50
79902                   16
99592                   65
gender_F               257
gender_M               312
race_asian              17
race_black              39
race_hispanic           15
race_white             375
age_at_least_75        161
height_at_least_70      69
weight_at_least_100     67
died                    99
d

In [38]:
%%time
(diagnosis[partition['validation']] == 1).any(level=0, axis=0).sum()

CPU times: user 68 ms, sys: 4 ms, total: 72 ms
Wall time: 72.1 ms


25000                   67
2724                    96
2762                    38
27651                    8
27652                   10
27800                   15
2859                    43
4019                   166
41071                   18
41401                   82
4240                    16
4241                    14
42731                   93
42732                   13
4275                    22
4280                    97
42832                    6
42833                   11
431                     17
5119                    24
51881                   77
5715                    10
5849                    73
5859                    36
78552                   27
79902                    7
99592                   42
gender_F               153
gender_M               191
race_asian              14
race_black              27
race_hispanic            8
race_white             240
age_at_least_75        103
height_at_least_70      50
weight_at_least_100     37
died                    54
d

In [3]:
%%time

dataset = {}
for part in ['train', 'validation']:
    I = partition[part]
    row_lengths = initialize.get_row_lengths(metadata[I])
    args = [metadata[I], sig_data[I], diagnosis[I], row_lengths]
    tensors = initialize.get_tensors(H, *args)
    dataset[part] = data_pipeline.build(H, tensors, part)

CPU times: user 17.7 s, sys: 228 ms, total: 17.9 s
Wall time: 17.9 s


In [8]:
x, y = next(iter(dataset['train']))

In [16]:
x['mask'][0]

<tf.Tensor: id=119680, shape=(8,), dtype=bool, numpy=array([False,  True, False,  True,  True, False,  True,  True])>

In [19]:
tf.expand_dims(tf.expand_dims(x['mask'], axis=1), axis=-1).shape

TensorShape([64, 1, 8, 1])

In [18]:
tf.reshape(x['mask'], [H['batch_size'], 1, len(H['input_sigs']), 1])

<tf.Tensor: id=119687, shape=(64, 8), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  Tru

In [13]:
tf.expand_dims(x['mask'], axis=1).shape

TensorShape([64, 1, 8])

In [9]:
x['mask'].shape

TensorShape([64, 8])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(64, 8)]            0                                            
__________________________________________________________________________________________________
signals (InputLayer)            [(64, 512, 8)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(64, 1, 8)]         0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_ExpandDims_3 (Tenso [(64, 512, 8, 1)]    0           signals[0][0]                    
______________________________________________________________________________________________

In [4]:
model = conv_model.build(H, diagnosis_codes=list(diagnosis))

model.fit(
    dataset['train'],
    validation_data = dataset['validation'],
    steps_per_epoch = 2**7,
    validation_steps = 1,
)

Train for 128 steps, validate for 1 steps
 19/128 [===>..........................] - ETA: 12:59 - loss: 1.3870 - pressure_loss: 1.6393 - diagnosis_loss: 1.3448 - pressure_ABP_systolic: 20.9199 - pressure_ABP_diastolic: 11.3560 - pressure_ABP_pulse: 18.6247 - pressure_CVP_systolic: 8.3876 - pressure_CVP_diastolic: 4.6429 - pressure_CVP_pulse: 8.3463 - pressure_ICP_systolic: 4.2087 - pressure_ICP_diastolic: 3.6083 - pressure_ICP_pulse: 4.4980 - pressure_PAP_systolic: 13.4451 - pressure_PAP_diastolic: 5.7991 - pressure_PAP_pulse: 12.1380 - pressure_hypertensive_sensitivity: 0.0000e+00 - pressure_hypertensive_specificity: 1.0000 - pressure_hypertensive_accuracy: 0.5000 - diagnosis_25000_diabetes_sensitivity: 0.5981 - diagnosis_25000_diabetes_specificity: 0.3357 - diagnosis_25000_diabetes_accuracy: 0.4669 - diagnosis_25000_diabetes_precise_sensitivity: 0.1222 - diagnosis_25000_diabetes_precise_threshold: 0.9980 - diagnosis_2724_hyperlipidemia_sensitivity: 0.6570 - diagnosis_2724_hyperlipide

KeyboardInterrupt: 